### Links
- pytorch cifar10 code - https://github.com/kuangliu/pytorch-cifar/blob/master/main.py


In [1]:
import argparse
import math
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms


In [2]:
print(torch.__version__)
print(torch.__file__)

1.0.1.post2
/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/__init__.py


In [3]:
# Source: https://github.com/kuangliu/pytorch-cifar/blob/master/models/mobilenet.py

class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
#         print(out.size())
        out = F.relu(self.bn2(self.conv2(out)))
#         print(out.size())
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [(48,2), (64,2), (128,2), (256,2), 256]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layers = self._make_layers(in_planes=16)
        self.linear = nn.Linear(256, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
#         print(out.size())
        out = self.layers(out)
#         print(out.size())
        out = F.avg_pool2d(out, 2)
#         print(out.size())
        out = out.view(out.size(0), -1)
#         print(out.size())
        out = self.linear(out)
#         print(out.size())
        out = F.log_softmax(out, dim=1)
        return out


In [4]:
parser = argparse.ArgumentParser(description='PyTorch CIFAR-10 Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')


_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='how many batches to wait before logging training status', metavar='N')

In [5]:
args = parser.parse_args(args=["--epochs=10"])
args

Namespace(batch_size=64, epochs=10, log_interval=10, lr=0.01, momentum=0.5, no_cuda=False, seed=1, test_batch_size=1000)

In [6]:
torch.manual_seed(args.seed)

In [7]:
device = torch.device("cpu")
device

device(type='cpu')

In [8]:
normalize = transforms.Normalize(
    mean=[0.4914, 0.4822, 0.4465],
    std=[0.2023, 0.1994, 0.2010],
)
train_transform = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])
valid_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

In [9]:
data_dir = "../data"

In [10]:
# load the dataset
train_dataset = datasets.CIFAR10(
    root=data_dir, train=True,
    download=True, transform=train_transform,
)

valid_dataset = datasets.CIFAR10(
    root=data_dir, train=True,
    download=True, transform=valid_transform,
)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
valid_size = 0.1
num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [12]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, sampler=train_sampler, num_workers=3)

test_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=args.test_batch_size, sampler=valid_sampler, num_workers=3)


In [13]:
model = MobileNet().to(device)
model

MobileNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Block(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Block(
      (conv1): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): B

In [14]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [15]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # to do: needed?
        optimizer.zero_grad()  # what does this do? zero-out any previous gradient info?
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            digits = int(math.log10(len(train_loader.dataset))) + 1
            print('Train Epoch: {} [{:05d}/{:05d} ({:02.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
            

In [16]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)
    
    # save the model!
    # source for timestamp formatting: https://stackoverflow.com/q/10607688/781938
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    # source for f-string: https://stackoverflow.com/a/42562236/781938
    torch.save(
        model.state_dict(),
        f'{timestamp}-cifar10_mobilenet-{epoch:05d}.pytorchmodel')


Train Epoch: 1 [00000/50000 (00%)]	Loss: 2.340225
Train Epoch: 1 [00640/50000 (01%)]	Loss: 2.287353
Train Epoch: 1 [01280/50000 (03%)]	Loss: 2.282860
Train Epoch: 1 [01920/50000 (04%)]	Loss: 2.280152
Train Epoch: 1 [02560/50000 (06%)]	Loss: 2.228048
Train Epoch: 1 [03200/50000 (07%)]	Loss: 2.229302
Train Epoch: 1 [03840/50000 (09%)]	Loss: 2.233533
Train Epoch: 1 [04480/50000 (10%)]	Loss: 2.199833
Train Epoch: 1 [05120/50000 (11%)]	Loss: 2.191037
Train Epoch: 1 [05760/50000 (13%)]	Loss: 2.157562
Train Epoch: 1 [06400/50000 (14%)]	Loss: 2.178512
Train Epoch: 1 [07040/50000 (16%)]	Loss: 2.144815
Train Epoch: 1 [07680/50000 (17%)]	Loss: 2.029194
Train Epoch: 1 [08320/50000 (18%)]	Loss: 2.047235
Train Epoch: 1 [08960/50000 (20%)]	Loss: 1.994708
Train Epoch: 1 [09600/50000 (21%)]	Loss: 2.058118
Train Epoch: 1 [10240/50000 (23%)]	Loss: 2.071334
Train Epoch: 1 [10880/50000 (24%)]	Loss: 1.958049
Train Epoch: 1 [11520/50000 (26%)]	Loss: 2.081695
Train Epoch: 1 [12160/50000 (27%)]	Loss: 1.914077


Train Epoch: 3 [12800/50000 (28%)]	Loss: 1.478503
Train Epoch: 3 [13440/50000 (30%)]	Loss: 1.377202
Train Epoch: 3 [14080/50000 (31%)]	Loss: 1.411937
Train Epoch: 3 [14720/50000 (33%)]	Loss: 1.270038
Train Epoch: 3 [15360/50000 (34%)]	Loss: 1.447731
Train Epoch: 3 [16000/50000 (36%)]	Loss: 1.282784
Train Epoch: 3 [16640/50000 (37%)]	Loss: 1.241104
Train Epoch: 3 [17280/50000 (38%)]	Loss: 1.344651
Train Epoch: 3 [17920/50000 (40%)]	Loss: 1.042982
Train Epoch: 3 [18560/50000 (41%)]	Loss: 1.555338
Train Epoch: 3 [19200/50000 (43%)]	Loss: 1.370060
Train Epoch: 3 [19840/50000 (44%)]	Loss: 1.501180
Train Epoch: 3 [20480/50000 (45%)]	Loss: 1.276277
Train Epoch: 3 [21120/50000 (47%)]	Loss: 1.517687
Train Epoch: 3 [21760/50000 (48%)]	Loss: 1.341361
Train Epoch: 3 [22400/50000 (50%)]	Loss: 1.449593
Train Epoch: 3 [23040/50000 (51%)]	Loss: 1.416455
Train Epoch: 3 [23680/50000 (53%)]	Loss: 1.258052
Train Epoch: 3 [24320/50000 (54%)]	Loss: 1.319130
Train Epoch: 3 [24960/50000 (55%)]	Loss: 1.248134


Train Epoch: 5 [25600/50000 (57%)]	Loss: 1.501039
Train Epoch: 5 [26240/50000 (58%)]	Loss: 1.026789
Train Epoch: 5 [26880/50000 (60%)]	Loss: 1.110331
Train Epoch: 5 [27520/50000 (61%)]	Loss: 1.239107
Train Epoch: 5 [28160/50000 (62%)]	Loss: 1.145940
Train Epoch: 5 [28800/50000 (64%)]	Loss: 1.434730
Train Epoch: 5 [29440/50000 (65%)]	Loss: 1.069602
Train Epoch: 5 [30080/50000 (67%)]	Loss: 1.149599
Train Epoch: 5 [30720/50000 (68%)]	Loss: 1.311656
Train Epoch: 5 [31360/50000 (70%)]	Loss: 1.117375
Train Epoch: 5 [32000/50000 (71%)]	Loss: 1.156481
Train Epoch: 5 [32640/50000 (72%)]	Loss: 1.059829
Train Epoch: 5 [33280/50000 (74%)]	Loss: 1.118913
Train Epoch: 5 [33920/50000 (75%)]	Loss: 1.134185
Train Epoch: 5 [34560/50000 (77%)]	Loss: 1.308957
Train Epoch: 5 [35200/50000 (78%)]	Loss: 1.174639
Train Epoch: 5 [35840/50000 (80%)]	Loss: 1.171293
Train Epoch: 5 [36480/50000 (81%)]	Loss: 0.947902
Train Epoch: 5 [37120/50000 (82%)]	Loss: 1.066269
Train Epoch: 5 [37760/50000 (84%)]	Loss: 1.106371


Train Epoch: 7 [38400/50000 (85%)]	Loss: 1.099205
Train Epoch: 7 [39040/50000 (87%)]	Loss: 1.056570
Train Epoch: 7 [39680/50000 (88%)]	Loss: 1.197369
Train Epoch: 7 [40320/50000 (89%)]	Loss: 1.144065
Train Epoch: 7 [40960/50000 (91%)]	Loss: 0.963556
Train Epoch: 7 [41600/50000 (92%)]	Loss: 1.010987
Train Epoch: 7 [42240/50000 (94%)]	Loss: 1.080479
Train Epoch: 7 [42880/50000 (95%)]	Loss: 1.172958
Train Epoch: 7 [43520/50000 (97%)]	Loss: 1.151149
Train Epoch: 7 [44160/50000 (98%)]	Loss: 1.065286
Train Epoch: 7 [44800/50000 (99%)]	Loss: 1.302160

Test set: Average loss: 0.1150, Accuracy: 2887/50000 (6%)

Train Epoch: 8 [00000/50000 (00%)]	Loss: 1.180228
Train Epoch: 8 [00640/50000 (01%)]	Loss: 0.937151
Train Epoch: 8 [01280/50000 (03%)]	Loss: 1.175508
Train Epoch: 8 [01920/50000 (04%)]	Loss: 1.134860
Train Epoch: 8 [02560/50000 (06%)]	Loss: 1.118706
Train Epoch: 8 [03200/50000 (07%)]	Loss: 1.074574
Train Epoch: 8 [03840/50000 (09%)]	Loss: 0.964571
Train Epoch: 8 [04480/50000 (10%)]	Loss:

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  Fi

KeyboardInterrupt: 

In [ ]:
torch.optim.Optimizer

In [ ]:
type(optimizer)

In [ ]:
isinstance(optimizer, torch.optim.SGD)

In [ ]:
issubclass(type(optimizer), torch.optim.Optimizer)